In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import QuantileTransformer

In [2]:
def get_score(true_values, predicted_values):
    mae_ = mean_absolute_error(true_values, predicted_values)
    mse_ = mean_squared_error(true_values, predicted_values)
    r2_ = r2_score(true_values, predicted_values)

    return mae_, mse_, r2_

# Loading data

### Using only values for FaceAPI without neutral gave the best result

In [3]:
dataset = pd.read_csv('dataset.csv')

In [4]:
dataset = dataset.loc[dataset['COND'] != '0']

data = dataset.iloc[:, :8]
labels = dataset.iloc[:, -2:]

In [5]:
data_train_full, data_test, labels_train_full, labels_test = train_test_split(data, labels, test_size=0.2, random_state=123)

data_train, data_validation, labels_train, labels_validation = train_test_split(data_train_full, labels_train_full, test_size=0.2, random_state=123)

data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 1]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 5]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 9]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 1]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 5]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 9]

data_train = data_train.loc[labels_train['ANS_VALENCE'] != 1]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 5]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 9]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 1]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 5]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 9]

# Scaling input values to Gaussian distributions

In [6]:
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
data_train_qt = quantile_transformer.fit_transform(data_train)
data_train_qt = pd.DataFrame(data_train_qt, columns=data_train.columns)
data_validation_qt = quantile_transformer.fit_transform(data_validation)
data_validation_qt = pd.DataFrame(data_validation_qt, columns=data_validation.columns)

# Training

In [10]:
svr = SVR(kernel='rbf')
mor = MultiOutputRegressor(svr)

mor.fit(data_train_qt, labels_train)
print('Training finished')

Training finished


In [11]:
predictions = mor.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation, predictions)
print(f'''Values for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

Values for validation set:
MAE: 1.991289270917886
MSE: 5.744849664668817
RMSE: 2.3968416019146566
R2:  -0.005691066562495961


# Using full dataset

In [12]:
dataset = pd.read_csv('dataset_full.csv')
dataset = dataset.loc[dataset['COND'] != '0']

data = dataset.iloc[:, :8]
labels = dataset.iloc[:, -2:]
data_train_full, data_test, labels_train_full, labels_test = train_test_split(data, labels, test_size=0.2,
                                                                              random_state=123)

data_train, data_validation, labels_train, labels_validation = train_test_split(data_train_full, labels_train_full,
                                                                                test_size=0.2, random_state=123)

data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 1]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 5]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 9]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 1]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 5]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 9]

data_train = data_train.loc[labels_train['ANS_VALENCE'] != 1]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 5]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 9]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 1]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 5]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 9]

In [13]:
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
data_train_qt = quantile_transformer.fit_transform(data_train)
data_train_qt = pd.DataFrame(data_train_qt, columns=data_train.columns)
data_validation_qt = quantile_transformer.fit_transform(data_validation)
data_validation_qt = pd.DataFrame(data_validation_qt, columns=data_validation.columns)

In [14]:
svr = SVR(kernel='rbf')
mor = MultiOutputRegressor(svr)

mor.fit(data_train_qt, labels_train)
print('Training finished')

Training finished


In [15]:
predictions = mor.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation, predictions)
print(f'''Values for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

Values for validation set:
MAE: 1.9981086516594242
MSE: 5.79187602767313
RMSE: 2.4066316767783826
R2:  0.0018144982502334983
